# Solving a 4x4 grid cell Sudoku with the latest D-Wave Quantum Annealer via Cloud (D-Wave leap)

In [1]:
# for classical solver (simulated annealing)
import dimod
import operator
import dwavebinarycsp
import numpy as np
import math
import os
import json

In [2]:
from datetime import datetime
from quantum_annealing_sudoku import quantum_annealing_sudoku
from quantum_annealing_sudoku.label_encoder import encode_var_labels, decode_var_labels

In [3]:
log_parent_dir = 'data/'
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = log_parent_dir+f"log_{start_time_str}"
os.mkdir(log_dir)
# This will generate a folder named something like log_14_04_2020__19_46_06

In [4]:
class QA_AnnealingParameters(): 
    embdding = {}
    num_reads = int 
    bqm = {}
    sampler_properties = {}
    result = {}
    
class SA_AnnealingParameters(): 
    random_sett = int
    num_reads = int 
    num_sweeps = int
    annealing_time = float
    sample_set = {}
        

# params for logging
params_qa = QA_AnnealingParameters()
params_sa = SA_AnnealingParameters()

In [5]:
sudoku_4x4 = quantum_annealing_sudoku.QuantumAnnealingSudoku(grid_9x9 = False)


#get some function shortcuts for better readability 
check_sudoku = sudoku_4x4.check_sudoku
encode_board_to_binary = sudoku_4x4.encode_board_to_binary
decode_board_from_binary = sudoku_4x4.decode_board_from_binary
print_board = sudoku_4x4.print_board

In [6]:
board = ((1,2,0,0),
         (0,0,0,0),
         (0,0,1,0),
         (0,0,4,0),
)
sudoku_4x4.print_board((board))

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [7]:
correct_board = ((1,2,3,4),
         (4,3,2,1),
         (2,1,4,3),
         (3,4,1,2),
)
sudoku_4x4.print_board(correct_board)

 1  2 | 3  4 
 4  3 | 2  1 
------|------
 2  1 | 4  3 
 3  4 | 1  2 



In [8]:
# check if the solution check works
print('Error Count: ',sudoku_4x4.check_sudoku(correct_board))
print('Error Count: ',sudoku_4x4.check_sudoku(board))

Error Count:  0
Error Count:  6


The QUBO for restricting the amount of numbers in a cell reads:

$$\alpha\sum_{i=1}^{N} \sum_{j=1}^{N} (\sum_{k=1}^{N} x_{ijk}-1)^2  $$

with N being number of rows, columns, and possible numbers.
$\alpha$ is a penalty weight. We set an offset of 1 in order to compensate the starting by zero whereas Sudoku board starts counting at 1.

In [9]:
constant = 0
N = 4
cell_qubo = {}
linear = {}
quad = {}
penalty_weight = 1
for i in range(1,N+1):
        for j in range(1,N+1):
            for k1 in range(1,N+1):  
                    var_1 = encode_var_labels(i,j,k1)
                    for k2 in range(1,N+1):
                            var_2 = encode_var_labels(i,j,k2)
                            if var_1 == var_2:
                                linear[var_1] = -1*penalty_weight
                            else:
                                quad[var_1,var_2]= 2*penalty_weight
                        #linear[var_1] = 2
                    
            constant+=1
            

cell_qubo[()] = constant*penalty_weight
cell_qubo["linear"] = linear
cell_qubo["quadratic"] = quad

#bqm = dimod.BinaryQuadraticModel(cell_qubo,constant,dimod.Vartype.BINARY)

Each column j can have not any duplicate number: 

$$\alpha\sum_{k=1}^{N} \sum_{j=1}^{N} (\sum_{i=1}^{N} x_{ijk}-1)^2  $$


In [10]:
penalty_weight=1
constant = 0
N = 4
column_qubo = {}
lin_column={}
quad_column={}
binary_board = sudoku_4x4.encode_board_to_binary(board)
for k in range(1,N+1):
        for j in range(1,N+1):
            for i1 in range(1,N+1):
                var_1 = encode_var_labels(i1,j,k)
                for i2 in range(1,N+1):               
                    var_2 = encode_var_labels(i2,j,k)
                    if var_1 == var_2:
                        lin_column[var_1] = -1*penalty_weight
                    else:
                        quad_column[var_1,var_2] = 2*penalty_weight
        constant+=1
        
column_qubo[()] = constant*penalty_weight
column_qubo['linear'] = lin_column
column_qubo['quadratic'] = quad_column

Each row i can have not any duplicate number: 

$$\alpha\sum_{k=1}^{N} \sum_{i=1}^{N} (\sum_{j=1}^{N} x_{ijk}-1)^2  $$


In [11]:
constant = 0
N = 4
row_qubo = {}
linear_row ={}
quadratic_row={}
penalty_weight=2
binary_board = sudoku_4x4.encode_board_to_binary(board)
for k in range(1,N):
        for i in range(1,N+1):
            for j1 in range(1,N+1):
                var_1 = encode_var_labels(i,j1,k)
                for j2 in range(1,N+1):      
                    var_2 = encode_var_labels(i,j2,k)
                    if var_1 == var_2:
                        linear_row[var_1] = -1*penalty_weight
                    else:
                        quadratic_row[var_1, var_2] = 2*penalty_weight
        constant+=1
        
row_qubo[()] = constant*penalty_weight
row_qubo['linear'] = linear_row
row_qubo['quadratic'] = quadratic_row

Initial numbers given as a hint cannot be changed: 

$$2\alpha\sum_{hint}(1- x_{ijk}) $$


In [12]:
binary_board = sudoku_4x4.encode_board_to_binary(board)
hint = []
penalty_weight = 4

for k, color in enumerate(binary_board):
    for i, row in enumerate(color):
        for j, cell in enumerate(row):
            if cell>0:
                hint.append([i+1,j+1,k+1])
                
hint_qubo_linear = {}
hint_qubo = {}
constant=2
for (i, j, k) in hint:
    re_label = encode_var_labels(i,j,k) 
    hint_qubo_linear[re_label] = -1*penalty_weight
    constant += 1

hint_qubo[()] = constant*penalty_weight
hint_qubo['linear'] = hint_qubo_linear

Each of the nine 3x3 sub-squares can not have a duplicate number:



$\alpha\sum_{k=1}^{N} (\sum_{j=1}^{2} \sum_{i=1}^{2} x_{(i+v)(j+v)k}-1)^2  $

$ u, v \in \{0,3,6\}$ : offset to each grid


In [13]:
from math import sqrt
constant = 0
N = 4
duplicate_qubo = {}
dupli_linear = {}
dupli_quadr = {}
sqrtN = int(sqrt(N))
for grid_i in range(sqrtN):
    for grid_j in range(sqrtN):
        for k in range(N):
            # there can be only one k in the same subgrid.
            for i1 in range(grid_i * 2, grid_i * 2 + 2):
                for j1 in range(grid_j * 2, grid_j * 2 + 2):
                    for i2 in range(grid_i * 2, grid_i * 2 + 2):
                        var_1 = encode_var_labels(i1+1,j1+1,k+1)
                        for j2 in range(grid_j * 2, grid_j * 2 + 2):
                            var_2 = encode_var_labels(i2+1,j2+1,k+1)
                            if var_1 == var_2:
                                dupli_linear[var_1] = -1
                            else:
                                dupli_quadr[var_1,var_2] = 1
            constant+=1

            
duplicate_qubo[()] = constant
duplicate_qubo['linear'] = dupli_linear
duplicate_qubo['quadratic'] = dupli_quadr

In [14]:
def append_linear(result_qubo_linear, qubo):    
    if not qubo.get('linear'):
        return result_qubo_linear
    for index, value in qubo['linear'].items():
        if result_qubo_linear.get(index):
            result_qubo_linear[index] += value     
        else:
            result_qubo_linear[index] = value
            
    return result_qubo_linear
            
def append_quadratic(result_qubo_quadratic, qubo):    
    if not qubo.get('quadratic'):
        return result_qubo_quadratic
    for index, value in qubo['quadratic'].items():
        if result_qubo_quadratic.get(index):
            result_qubo_quadratic[index] += value 
        else:
            result_qubo_quadratic[index] = value
            
    return result_qubo_quadratic
            

In [15]:
# qubos that are needed to achieve the constraints
qubos=[cell_qubo, column_qubo, row_qubo, hint_qubo, duplicate_qubo]

The final QUBO reads:

$$ Q_{final} = \alpha\sum_{i=1}^{N} \sum_{j=1}^{N} (\sum_{k=1}^{N} x_{ijk}-1)^2 + \beta\sum_{k=1}^{N} \sum_{j=1}^{N} (\sum_{i=1}^{N} x_{ijk}-1)^2 
+ \gamma\sum_{k=1}^{N} \sum_{i=1}^{N} (\sum_{j=1}^{N} x_{ijk}-1)^2 
+ 2\theta\sum_{hint}(1- x_{ijk}) 
\\ + \epsilon\sum_{k=1}^{N} (\sum_{j=1}^{2} \sum_{i=1}^{2} x_{(i+v)(j+v)k}-1)^2$$



With $x_{ijk} \in \{0,1\}$ and $i,j,k \in \{1..9\}$

In [16]:
result_qubo_linear = {}
result_qubo_quadratic = {}
for qubo in qubos:    
    if qubo is not None:
        result_qubo_linear = append_linear(result_qubo_linear, qubo)
        result_qubo_quadratic = append_quadratic(result_qubo_quadratic, qubo)

In [17]:
constant= cell_qubo.get((),0) + hint_qubo.get((),0) + row_qubo.get((),0) + column_qubo.get((),0) + duplicate_qubo.get((),0)
bqm = dimod.BinaryQuadraticModel(result_qubo_linear,result_qubo_quadratic, constant,dimod.Vartype.BINARY)

In [18]:
# test run
import neal
sampler = neal.SimulatedAnnealingSampler()
sample_set=sampler.sample(bqm, num_reads=30,num_sweeps=2222)


In [19]:
from random import randrange
import time

def find_optimal_solution(best_solution_global, current_solution, current_energy, 
                          sa_sample_set):
    best_solution = {}
    error_count=20
    iteration = 0
    sampler = neal.SimulatedAnnealingSampler()
    start_time = time.perf_counter()
    while error_count>0:
        """
        sample_set = sampler.sample(bqm, seed=1234, beta_range=[0.1, 4.2],
                                        num_reads=10, num_sweeps=20000,
                                       beta_schedule_type='geometric')
        """
        random_seed = randrange(9000)
        #random_seed = 5832
        #num_reads = randrange(12000)
        num_sweeps = randrange(6000)
        num_reads = 1000 
        #num_sweeps= 50
        #num_sweeps*=iteration
        pre_anneal = time.perf_counter()
        sample_set=sampler.sample(bqm,seed=random_seed ,num_reads=num_reads,num_sweeps=num_sweeps)
        post_anneal = time.perf_counter()
        annealing_time = post_anneal-pre_anneal
        iteration+=1
        sa_sample = sample_set.copy()
        sa_sample.info.update({'annealing_time':annealing_time})
        sa_sample_set.append(sa_sample)
        
        params_sa.num_reads = num_reads
        params_sa.num_sweeps = num_sweeps
        params_sa.random_seed = random_seed
        params_sa.annealing_time = annealing_time
        params_sa.sa_sample_set = sample_set.to_serializable()
        
        sorted_params_dict_sa = {k: params_sa.__dict__[k] for k 
                      in sorted(params_sa.__dict__.keys())}
        
        lowest_energy = sample_set.first.energy
        
        params_filepath_sa = os.path.join(log_dir, 'params_sa_'+str(iteration)+'_'+str(lowest_energy)+"_"+str(num_reads)+'.json')
        json.dump(sorted_params_dict_sa, open(params_filepath_sa, 'w'), indent=4)


        for solution, energy in sample_set.data(['sample', 'energy']):
            binary_solution_board= np.zeros((4, 4, 4))
            for index, value in solution.items():
                if type(index) is int and index>0:
                    board_index = decode_var_labels(index)
                    binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
            solution_board=decode_board_from_binary(binary_solution_board)
            error_count_temp = check_sudoku(solution_board)
            
            
            
            current_solution.append(1)
            current_energy.append(energy)
            current_solution[:] = []
            current_energy[:] = []
            current_solution.append(solution)
            current_energy.append(energy)
            
            
        overall_time = time.perf_counter()-start_time
        if error_count_temp<error_count:
            best_solution = solution_board
            best_solution_global.append(solution)
            error_count=error_count_temp
            print("\nError Count:",error_count, "iteration:", iteration, "Energy:", energy,
                  "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps, 
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            
        if iteration%100 == 0:
            print("\nCurrent State:",
                  "Error Count:",error_count_temp, "iteration:", iteration, "Energy:", energy,
                 "Seed:", random_seed, "num_reads:", num_reads, "sweeps:", num_sweeps,
                 "Annealing Time:", annealing_time, "Overall Time:", overall_time)
            

            
    return sample_set
        

In [20]:
import multiprocessing
manager = multiprocessing.Manager()
best_solution = manager.list()
current_solution = manager.list()
current_energy = manager.list()
sa_sample_set = manager.list()

process = multiprocessing.Process(target=find_optimal_solution, 
            
                                  args= (best_solution, current_solution, 
                                         current_energy, sa_sample_set))
process.start()




Error Count: 3 iteration: 1 Energy: -10.0 Seed: 2151 num_reads: 1000 sweeps: 1461 Annealing Time: 1.741178756001318 Overall Time: 3.886241633001191

Error Count: 0 iteration: 2 Energy: -10.0 Seed: 8688 num_reads: 1000 sweeps: 3958 Annealing Time: 4.681738432998827 Overall Time: 10.610220945000037


In [21]:
import time
time.sleep(30)
#wait with process terminating; in order to have  
#results when doing a (quick) autorun

process.terminate()
process

<Process name='Process-2' pid=13163 parent=13133 stopped exitcode=0>

In [22]:
#sample_set = list(current_solution)
sample_set_simulated_annealing = list(best_solution)

In [23]:
sa_sample_set[0].info

{'beta_range': [0.02829172165550797, 9.210340371976184],
 'beta_schedule_type': 'geometric',
 'annealing_time': 1.741178756001318}

In [24]:
solution={}
solution_2={}
solution_3={}
for number, solution in enumerate(sample_set_simulated_annealing):
    if number==0:
        solution = solution
    elif number==1:
        solution_2 = solution
    elif number==2:
        solution_3 = solution
        
    
last_solution = solution

In [25]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value


In [26]:
solution_board=decode_board_from_binary(binary_solution_board)
solution_board

array([[2., 3., 1., 4.],
       [1., 4., 2., 3.],
       [4., 2., 3., 1.],
       [3., 1., 4., 2.]])

In [27]:
print_board(board)

 1  2 | -  - 
 -  - | -  - 
------|------
 -  - | 1  - 
 -  - | 4  - 



In [28]:
sudoku_4x4.print_board(solution_board)
print("Error Count:",sudoku_4x4.check_sudoku(solution_board))
print("BQM Energy:", bqm.energy(solution))

 2  3 | 1  4 
 1  4 | 2  3 
------|------
 4  2 | 3  1 
 3  1 | 4  2 

Error Count: 0
BQM Energy: -10.0


In [29]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in solution_2.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
sudoku_4x4.print_board(solution_board)
print("Error Count:",sudoku_4x4.check_sudoku(solution_board))

 2  3 | 1  4 
 1  4 | 2  3 
------|------
 4  2 | 3  1 
 3  1 | 4  2 

Error Count: 0


In [30]:
binary_solution_board= np.zeros((9, 9, 9))
for index, value in solution_3.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 2  3 | 1  4 
 1  4 | 2  3 
------|------
 4  2 | 3  1 
 3  1 | 4  2 

Error Count: 0


In [31]:
bqm.energy(solution)

-10.0

In [32]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in sa_sample_set[0].first.sample.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
print("Error Count:",check_sudoku(solution_board))

 2  3 | 1  4 
 1  4 | 2  3 
------|------
 4  2 | 3  1 
 3  1 | 4  2 

Error Count: 0


In [33]:
sa_sample_df = sa_sample_set[0].to_pandas_dataframe()
annealing_time = sa_sample_set[0].info['annealing_time']
sa_sample_df['annealing_time'] = annealing_time

In [34]:
sa_sample_df.head(2)

,1,2,3,4,10,11,12,13,19,20,...,263,264,265,271,272,273,274,energy,num_occurrences,annealing_time
0,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,-22.0,1,1.741179
1,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,-22.0,1,1.741179


In [35]:
sa_sample_df.to_csv(log_dir+
                    '/simulated_annealing_'+ str(annealing_time)+
                   '.csv')

In [36]:
bqm_graph = bqm.to_networkx_graph()

In [37]:
print('Number of problem nodes (variables):',len(bqm_graph.nodes))
print('Number of problem edges (couplings):',len(bqm_graph.edges))

Number of problem nodes (variables): 64
Number of problem edges (couplings): 304


Turns out the number of edges and nodes is feasible (small) enough to be embedded on the quantum annealing hardware!

In [38]:
# imports for D-Wave
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dwave.embedding import embed_bqm, unembed_sampleset
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from dwave.embedding.chain_breaks import majority_vote

import dwave.inspector


#### Either use the D-Wave embedding composite to ansemble the steps of embedding and sampling or split them up by using minorminor.find_embedding

In [39]:
use_embedding_composite=False

In [40]:
# Use a D-Wave system as the sampler
sampler = DWaveSampler() 
print("QPU {} was selected.".format(sampler.solver.name))

QPU Advantage_system1.1 was selected.


In [41]:
embedding = {}
if use_embedding_composite is True: 
    # Set up a D-Wave system EmbeddingComposite as the sampler
    sampler = EmbeddingComposite(sampler)

else:
    # split sampling into embedding first and then sampling
    __, target_edgelist, target_adjacency = sampler.structure
    #bqm = BinaryQuadraticModel.from_qubo(Q)
    emb = find_embedding(bqm.to_qubo()[0], target_edgelist)
    embedded_bqm = embed_bqm(source_bqm=bqm,embedding=emb ,target_adjacency=target_adjacency)
    embedding['embedding'] = emb

In [42]:
# sampling on leap hardware
# num_read is very important as it defines the number of sampling 
# made on the hardware

NUM_READS=1000
if use_embedding_composite is True:
    result = sampler.sample(bqm, num_reads=NUM_READS)
    d_wave_solution=result.first.sample
else:
    result = sampler.sample(embedded_bqm, num_reads=NUM_READS)
    unembedded_result = unembed_sampleset(result, emb, bqm, chain_break_method=majority_vote)
    d_wave_solution = unembedded_result.first.sample


In [43]:
result_df = result.to_pandas_dataframe()
result_df.to_csv(log_dir
                 +'/annealing_run'+result.info['problem_id']+'.csv')

In [44]:
params_qa = QA_AnnealingParameters()
params_qa.num_reads = NUM_READS
params_qa.bqm = bqm.to_serializable()
params_qa.embdding = embedding
params_qa.sampler_properties = sampler.properties
params_qa.result = result.to_serializable()
sorted_params_dict_qa = {k: params_qa.__dict__[k] for k 
                      in sorted(params_qa.__dict__.keys())}

params_filepath_qa = os.path.join(log_dir,'params_qa'+str(NUM_READS)+"_"+result.info['problem_id']+'.json')
json.dump(sorted_params_dict_qa, open(params_filepath_qa, 'w'), indent=4)

In [45]:
binary_solution_board= np.zeros((4, 4, 4))
for index, value in d_wave_solution.items():
    if type(index) is int and index>0:
        board_index = decode_var_labels(index)
        binary_solution_board[board_index[2]-1][board_index[0]-1][board_index[1]-1] = value
print_board(solution_board)
error_count = check_sudoku(solution_board)
print("Error Count:", error_count)

 2  3 | 1  4 
 1  4 | 2  3 
------|------
 4  2 | 3  1 
 3  1 | 4  2 

Error Count: 0


In [46]:
sample_anneal_time = result.info['timing']['qpu_anneal_time_per_sample']
if error_count==0:
    print('The quantum annealer found a\
    valid solution in {} miliseconds !'.format(sample_anneal_time))
else:
    print('The quantum annealer DID NOT find a\
valid solution and needed {} miliseconds ! Error Count: {}'.format(sample_anneal_time, error_count))

The quantum annealer found a    valid solution in 20 miliseconds !


### It is possible to solve a 4x4 Sudoku with the Quantum Annealer in constant time!